In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import re
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import TweetTokenizer
import string
from sklearn.utils import shuffle
from scipy.stats import mode
from numpy import mean
from numpy import median
from numpy import std
import math
from collections import defaultdict
from collections import Counter
import nltk
import numpy as np
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet
from nltk.corpus import sentiwordnet as swn
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn import svm
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse as sps
from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords
from textblob import TextBlob
from textblob.taggers import PatternTagger
from sklearn.cross_validation import KFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import OneHotEncoder

In [2]:
pos_dict = {"CC":1, "CD":2, "DT":3, "EX":4, "FW":5,"IN":6,"JJ":7,
         "JJR":8, "JJS":9, "MD":10, "NN":11, "NNS":12, "NNP":13,
         "NNPS":14,"PDT":15, "POS":16, "PRP":17, "PRP$":18, "RB":19,
         "RBR":20, "RBS":21,"RP":22,"SYM":23,"TO":24,"UH":25,"VB":26,
         "VBD":27,"VBG":28, "VBN":29, "VBP":30, "VBZ":31, "WDT":32,
          "WP":33, "WP$":34, "WRB":35, 'NN|JJ':36}

In [3]:
def get_set(f):
    s = set()
    set_file = open(f, "r")
    for line in set_file:
        l = line.split()
        s.add(l[0])
    return s

In [4]:
swear_words_set = get_set("sp_files/swear_words_list.txt")

In [5]:
def get_data_set(filename):
    data = pd.read_csv(filename, encoding="utf-8", engine='python')
    data = data[["text", "sarc"]]
    data = data[data["text"].notnull()]
    data = data[data["sarc"].notnull()]
    data["sarc"] = data["sarc"].apply(lambda x: int(x))
    data = data.drop_duplicates()
    sarc = data[data["sarc"]==1]
    not_sarc = data[data["sarc"]==0][:len(data[data["sarc"]==1])]
    data_1_1 = pd.concat([sarc, not_sarc], ignore_index=True)
    data_1_1 = shuffle(data_1_1)
    return data_1_1

In [6]:
data = get_data_set("sarcasm_set.csv")
data_small = get_data_set("sarcasm_set_small.csv")

In [7]:
affect = pd.read_csv("sp_files/Ratings_Warriner_et_al-2.csv")
keys = list(affect["Word"])
values = list(affect["V.Mean.Sum"])
affect_scores = dict(zip(keys, values))

In [8]:
def get_anc_dict(f):
    anc = defaultdict(dict)
    anc_file = open(f, "r")
    for line in anc_file:
        l = line.split()
        anc[l[0]][l[2]] = int(l[3]) 
    return anc

In [9]:
def get_strength_dict(f):
    s_d = {}
    s_file = open(f, "r")
    for line in s_file:
        l = line.split()
        s_d[re.sub(r"\*",r"",l[0])] = int(l[1]) 
    return s_d

In [10]:
booster_dict = get_strength_dict("sp_files/BoosterWordList.txt")

In [11]:
sentiments_dict = get_strength_dict("sp_files/SentimentLookupTable.txt")

In [12]:
anc_all_data = "sp_files/ANC-all-lemma.txt"
anc_all = get_anc_dict(anc_all_data)

In [13]:
def get_tagging(text):
    text_and_tags = PatternTagger().tag(text.decode("utf-8").lower())
    return text_and_tags

In [14]:
def get_distrbution(arr, n, k):
    d = [0] * n
    for i in arr: 
        d[int(round(i)) + k] += 1
    return d

In [15]:
def get_distribution_f(df, col):
    df = df.copy()
    df[col+"_mean"]= df[col].apply(lambda x: mean(x or [0]))
    df[col+"_median"]= df[col].apply(lambda x: median(x or [0]))
    df[col+"_mode"]= df[col].apply(lambda x: mode(x or [0],axis=None).mode[0])
    df[col+"_std"]= df[col].apply(lambda x: std(x or [0]))
    df[col+"_max"]= df[col].apply(lambda x: max(x or [0]))
    return df

In [16]:
def get_sentiment(x):
    s = [senti_score(i[0]) for i in x]
    return max(s or [0]) - min(s or [0])

def get_affect_score(x):
    s = [affect_scores[i[0]] for i in x if i[0] in affect_scores]
    return max(s or [0]) - min(s or [0])

def get_contrasting_features(df):
    df = df.copy()
    df["del_sentiment"] = df["tagged_words"].apply(lambda x: get_sentiment(x))
    df["del_affect"] = df["tagged_words"].apply(lambda x: get_affect_score(x))
    return df

In [17]:
def get_multi_f(df, col,n):
    df = df.copy()
    for i in range(n):
        df[col + "_" + str(i)] = df[col].apply(lambda x: x[i])
    df = df.drop(col, axis=1)
    return df

In [18]:
def check_swear_words(t):
    for w in t:
        if w[0] in swear_words_set:
            return 1
    return 0

In [19]:
def get_mood_f(df):
    df = df.copy()
    df["sen_d"] = df["tagged_words"].apply(lambda x: [senti_score(i[0]) for i in x])
    df["aff_d"] = df["tagged_words"].apply(lambda x: [affect_scores[i[0]] for i in x if i[0] in affect_scores])
    df = get_distribution_f(df, "sen_d")
    df = get_distribution_f(df, "aff_d")
    df["d_s"] = df["sen_d"].apply(lambda x: get_distrbution([i for i in x], 11, 5))
    df["d_a"] = df["aff_d"].apply(lambda x: get_distrbution(x, 9, -1))
    df = get_multi_f(df, "d_s", 11)
    df = get_multi_f(df, "d_a",9)
    df["num_sen_words"] = df["sen_d"].apply(lambda x: len([i for i in x if i != 0]))
    df["num_af_words"] = df["aff_d"].apply(lambda x: len(x))
    df["sen_score"] = df["text"].apply(lambda x: TextBlob(x.decode("utf-8").lower()).polarity)
    df["swear_words"] = df["tagged_words"].apply(lambda x: check_swear_words(x))
    return df

In [20]:
def check_a_n(x):
    for i in x:
        if any(char.isdigit()for char in i[0]) and any(char.isalpha()for char in i[0]):
            return 1
    return 0


def check_not_vowel(x):
    vowels = {"a", "e", "i", "o", "u", "A", "E", "I", "O", "U"}
    for i in x:
        if not any(char in vowels for char in i[0]):
            return 1 
    return 0


def get_pr_dict_words(x):
    count = 0
    for word in x:
        if word[0] in anc_all:
            count +=1
    return count * 1.0 / len(x)

In [21]:
def get_punctuation_d(x):
    n_1 = x.count("!")
    n_2 = x.count(".")
    n_3 = x.count(",")
    n_4 = x.count('"')
    n_5 = x.count("'")
    n_6 = x.count("*")
    n_7 = x.count("?")
    return [n_1, n_2, n_3, n_4, n_5, n_6, n_7]

In [22]:
def get_grammar_f(df):
    df = df.copy()
    df["pos_d"] = df["tagged_words"].apply(lambda x: [pos_dict[i[1]] for i in x])
    df["pos_d"] = df["pos_d"].apply(lambda x:get_distrbution(x, 36, -1))
    df = get_multi_f(df, "pos_d", 36)
    df["alphanumeric_words"] = df["tagged_words"].apply(lambda x: check_a_n(x))
    df["w_o_vowels"] = df["tagged_words"].apply(lambda x: check_not_vowel(x))
    df["pr_dict_word"] = df["tagged_words"].apply(lambda x: get_pr_dict_words(x))
    return df

In [23]:
def senti_score(x):
    wl = WordNetLemmatizer()
    ps = PorterStemmer()
    if x in sentiments_dict:
        return sentiments_dict[x]
    lemma  = wl.lemmatize(x)
    if lemma in sentiments_dict:
        return sentiments_dict[lemma]
    stem = ps.stem(x)
    if stem in sentiments_dict:
        return sentiments_dict[stem]
    return 0.0

In [24]:
def check_repeated_chars(text):
    for i in range(len(text)-2):
        if (text[i] == text[i+1]) and (text[i] == text[i+2]):
            return 1
    return 0


def check_repeated_in_sen(tagged_text):
    for w in tagged_text:
        if TextBlob(w[0] + " ").polarity != 0:
            if check_repeated_chars(w[0]):
                return 1
    return 0

In [25]:
def get_written_features(df):
    df = df.copy()
    df["rep_chars"] = df["text"].apply(lambda x: check_repeated_chars(x))
    df["rep_chars_in_sen"] = df["tagged_words"].apply(lambda x: check_repeated_in_sen(x))
    df["original_tagged_words"] = df["text"].apply(lambda x: TextBlob(x.decode("utf-8"),pos_tagger=PatternTagger()).tags)
    df["cap_words_number"] = df["original_tagged_words"].apply(lambda x: len([i for i in x if i[0].isupper()]))
    df["cap_pos_d"] = df["original_tagged_words"].apply(lambda x: [pos_dict[i[1]] for i in x if i[0].isupper()])
    df["cap_pos_d"] = df["cap_pos_d"].apply(lambda x:get_distrbution(x, 36, -1))
    df = get_multi_f(df, "cap_pos_d", 36)
    df["punct_dist"] = df["text"].apply(lambda x: get_punctuation_d(x))
    df = get_multi_f(df, "punct_dist", 7)
    return df

In [26]:
def get_first_sent_l_word(x):
    for ind, i in enumerate(x):
        if TextBlob(i[0]+ " ").polarity != 0:
            return ind + 1
    return 0

def get_first_a_l_word(x):
    for ind, i in enumerate(x):
        if i[0] in affect_scores:
            return ind + 1
    return 0

def get_density(x):
    verbs = re.compile('VB*')
    nouns = re.compile('NN*')
    adjectives = re.compile('JJ*')
    adverbs = re.compile('RB*')
    l = [i for i in x if verbs.match(i[1]) or nouns.match(i[1]) or adjectives.match(i[1]) or
         adverbs.match(i[1])]
    return len(l) * 1.0 / len(x)

In [27]:
def get_structural_features(df):
    df = df.copy()
    df["pos_1_w"] = df["tagged_words"].apply(lambda x: pos_dict[x[0][1]])
    df = pd.concat([df, pd.get_dummies(df["pos_1_w"], prefix="pos_1_w_" )], axis=1)
    df = df.drop("pos_1_w",axis=1)
    df["pos_2_w"] = df["tagged_words"].apply(lambda x: pos_dict[x[1][1]])
    df = pd.concat([df, pd.get_dummies(df["pos_2_w"], prefix="pos_2_w_" )], axis=1)
    df = df.drop("pos_2_w",axis=1)
    df["pos_3_w"] = df["tagged_words"].apply(lambda x: pos_dict[x[2][1]])
    df = pd.concat([df, pd.get_dummies(df["pos_3_w"], prefix="pos_3_w_" )], axis=1)
    df = df.drop("pos_3_w",axis=1)
    df["pos_l1_w"] = df["tagged_words"].apply(lambda x: pos_dict[x[len(x)-1][1]])
    df = pd.concat([df, pd.get_dummies(df["pos_l1_w"], prefix="pos_l1_w_" )], axis=1)
    df = df.drop("pos_l1_w",axis=1)
    df["pos_l2_w"] = df["tagged_words"].apply(lambda x: pos_dict[x[len(x)-2][1]])
    df = pd.concat([df, pd.get_dummies(df["pos_l2_w"], prefix="pos_l2_w_" )], axis=1)
    df = df.drop("pos_l2_w",axis=1)
    df["pos_l3_w"] = df["tagged_words"].apply(lambda x: pos_dict[x[len(x)-3][1]])
    df = pd.concat([df, pd.get_dummies(df["pos_l3_w"], prefix="pos_l3_w_" )], axis=1)
    df = df.drop("pos_l3_w",axis=1)
    df["p_1_s"] = df["tagged_words"].apply(lambda x: get_first_sent_l_word(x))
    df["p_1_a"] = df["tagged_words"].apply(lambda x: get_first_a_l_word(x))
    df["density"] = df["tagged_words"].apply(lambda x: get_density(x))
    df["intens"] = df["tagged_words"].apply(lambda x: len([i for i in x if i[0] in booster_dict]))
    df["pronouns"] = df["tagged_words"].apply(lambda x: len([i for i in x if i[1] == "PRP"]))
    return df

In [28]:
def create_features(df):
    df = df.copy()
    df["tagged_text"] = df["text"].apply(lambda x: get_tagging(x))
    df["tagged_words"] = df["text"].apply(lambda x: TextBlob(x.encode("utf-8").lower(),pos_tagger=PatternTagger()).tags)
    df = get_contrasting_features(df)
    df["length_d"] = df["tagged_words"].apply(lambda x: [len(i[0]) for i in x])
    df = get_distribution_f(df, "length_d")
    df = get_mood_f(df)
    df = get_grammar_f(df)
    df = get_written_features(df)
    df = get_structural_features(df)
    return df

In [29]:
data_f_small = create_features(data_small)

In [30]:
features = list(data_f_small.columns.difference([u'text', u'sarc', u'tagged_text',
                          u'tagged_words','length_d','sen_d', 'aff_d', 'original_tagged_words']))

In [65]:
def get_trains_tests(data_f):
    print "w"
    skf = KFold(len(data_f), n_folds=10, shuffle=False, random_state=None)
    trains = []
    tests = []
    data_1_1 = shuffle(data_f)
    i = 0
    for train, test in skf:
        trains.append(data_f.iloc[train])
        data_f.iloc[train][["text", "sarc"]].to_csv("data_train_kf_"+str(i)+".csv", encoding='utf-8',index=False)
        tests.append(data_f.iloc[test])
        data_f.iloc[test][["text", "sarc"]].to_csv("data_test_kf_"+str(i)+".csv", encoding='utf-8',index=False)
        i += 1
    return trains, tests

In [66]:
def get_scores(trains, tests, clf, features):
    f_1_scores = []
    roc_auc_scores = []
    recall_scores = []
    precision_scores = []
    accuracy_scores = []
    confusion_matrixes = []
    y_prs = []
    y_probas = []
    i = 0
    for train, test in tqdm(zip(trains,tests)):
        scaler = preprocessing.StandardScaler()
        x_train = scaler.fit_transform(train[features].astype(float))
        y_train = train.sarc
        x_test = scaler.transform(test[features].astype(float))
        y_test = test.sarc
        clf.fit(x_train, y_train)
        y_pr = clf.predict(x_test)
        
        y_proba = clf.predict_proba(x_test)
        pd.Series(list(y_pr)).to_csv("bm_y_prd_kf_"+str(i)+".csv")
        pd.Series(list(y_proba[:,1])).to_csv("bm_y_prb_kf_"+str(i)+".csv")
        i += 1
        y_prs.append(y_pr)
        y_probas.append(y_proba)
        f_1_scores.append(f1_score(y_test, y_pr))
        roc_auc_scores.append(roc_auc_score(y_test, y_proba[:,1]))
        accuracy_scores.append(accuracy_score(y_test, y_pr))
        confusion_matrixes.append(confusion_matrix(y_test, y_pr))
        recall_scores.append(recall_score(y_test, y_pr))
        precision_scores.append(precision_score(y_test, y_pr))
    return y_prs, y_probas, f_1_scores, roc_auc_scores, recall_scores,\
          precision_scores, accuracy_scores, confusion_matrixes

In [67]:
trains_small, tests_small = get_trains_tests(data_f_small)

w


In [68]:
clf = RandomForestClassifier(n_estimators=550, n_jobs=-1,random_state=20)
f_rf_prd, f_rf_prb, f1_small, auc_small, rec_small, pr_small, acc_small, con_small = get_scores(trains_small,\
                                                                                                tests_small, clf,features)

In [69]:
print "avg f1:", np.mean(f1_small)
print "avg roc_auc:", np.mean(auc_small)
print "avg accuracy:", np.mean(acc_small)
print "avg precision:", np.mean(pr_small)
print "avg recall:", np.mean(rec_small)

avg f1: 0.745597540542
avg roc_auc: 0.805304048707
avg accuracy: 0.723020145016
avg precision: 0.689175936033
avg recall: 0.81292847442


In [205]:
clf = svm.SVC(C=1.5, probability=True)
f_svm_prd, f_svm_prb, f1_small, auc_small, rec_small, pr_small, acc_small, con_small = get_scores(trains_small, tests_small, clf, features)

In [206]:
print "avg f1:", np.mean(f1_small)
print "avg roc_auc:", np.mean(auc_small)
print "avg accuracy:", np.mean(acc_small)
print "avg precision:", np.mean(pr_small)
print "avg recall:", np.mean(rec_small)

avg f1: 0.722693748569
avg roc_auc: 0.792003697685
avg accuracy: 0.714771201793
avg precision: 0.703474185489
avg recall: 0.744514558464


In [70]:
def get_bag_of_words_test_trains(trains_data, tests_data):
    trains = []
    tests = []
    y_trains = []
    y_tests = []
    for train, test in zip(trains_data, tests_data):
        trigram_vectorizer = CountVectorizer(ngram_range=(1, 3), min_df=1)
        X_train = trigram_vectorizer.fit_transform(train["text"])
        X_test = trigram_vectorizer.transform(test["text"])
        trains.append(X_train)
        tests.append(X_test)
        y_trains.append(train["sarc"])
        y_tests.append(test["sarc"])
    return trains, tests, y_trains, y_tests

In [75]:
def get_scores_bag(trains, tests, y_trains, y_tests, clf):
    f_1_scores = []
    roc_auc_scores = []
    recall_scores = []
    precision_scores = []
    accuracy_scores = []
    confusion_matrixes = []
    y_pr_trains = []
    y_proba_trains = []
    y_prs = []
    y_probas = []
    i = 0
    for train, test, y_train, y_test in tqdm(zip(trains, tests, y_trains, y_tests)):
        clf.fit(train, y_train)
        y_pr_train = clf.predict(train)
        y_pr_trains.append(y_pr_train)
        y_proba_train = clf.predict_proba(train)
        y_proba_trains.append(y_proba_train[:,1])
        y_pr = clf.predict(test)
        y_prs.append(y_pr)
        y_proba = clf.predict_proba(test)
        pd.Series(list(y_pr)).to_csv("bag_y_prd_kf_"+str(i)+".csv")
        pd.Series(list(y_proba[:,1])).to_csv("bag_y_prb_kf_"+str(i)+".csv")
        y_probas.append(y_proba[:,1])
        f_1_scores.append(f1_score(y_test, y_pr))
        roc_auc_scores.append(roc_auc_score(y_test, y_proba[:,1]))
        accuracy_scores.append(accuracy_score(y_test, y_pr))
        confusion_matrixes.append(confusion_matrix(y_test, y_pr))
        recall_scores.append(recall_score(y_test, y_pr))
        precision_scores.append(precision_score(y_test, y_pr))
        i += 1
    return y_pr_trains, y_proba_trains, y_prs, y_probas, f_1_scores, roc_auc_scores,\
           recall_scores, precision_scores, accuracy_scores, confusion_matrixes

In [72]:
bag_trains, bag_tests, y_trains, y_tests = get_bag_of_words_test_trains(trains_small, tests_small)

In [76]:
clf = LogisticRegression(C=0.5)
log_y_tr_prd, log_y_tr_prb, log_y_tst_prd, log_y_tst_prb, f1_small_2, auc_small_2, rec_small_2, pr_small_2,\
acc_small_2, con_small_2 = get_scores_bag(bag_trains, bag_tests, y_trains, y_tests, clf)

In [77]:
print "avg f1:", np.mean(f1_small_2)
print "avg roc_auc:", np.mean(auc_small_2)
print "avg accuracy:", np.mean(acc_small_2)
print "avg precision:", np.mean(pr_small_2)
print "avg recall:", np.mean(rec_small_2)

avg f1: 0.735144037597
avg roc_auc: 0.812584746875
avg accuracy: 0.735705379679
avg precision: 0.736478061837
avg recall: 0.734311108627


In [167]:
def combine_data(trains, tests, y_tr_prds, y_tr_prbs, y_tst_prs, y_tst_prbs):
    new_trains = []
    new_tests = []
    for tr, tst, y_tr_prd, y_tr_prb, y_tst_prd, y_tst_prb in zip(trains, tests, y_tr_prds,\
                                                                   y_tr_prbs, y_tst_prs, y_tst_prbs):
        new_tr = tr.copy()
        new_tst = tst.copy()
        new_tr["y_prd"] = y_tr_prd
        new_tr["y_prb"] = y_tr_prb
        new_tst["y_prd"] = y_tst_prd
        new_tst["y_prb"] = y_tst_prb
        new_trains.append(new_tr)
        new_tests.append(new_tst)
    return new_trains, new_tests

In [216]:
data_c_trains, data_c_tests = combine_data(trains_small, tests_small, y_tr_prd, y_tr_prb, y_tst_prd, y_tst_prb)

In [217]:
clf = RandomForestClassifier(n_estimators=550, n_jobs=-1,random_state=20)
m_prd, m_prb, mf1_small, mauc_small, mrec_small, mpr_small, macc_small, mcon_small = get_scores(data_c_trains, data_c_tests, clf, features + ["y_prd", "y_prb"])

In [218]:
print "avg f1:", np.mean(mf1_small)
print "avg roc_auc:", np.mean(mauc_small)
print "avg accuracy:", np.mean(macc_small)
print "avg precision:", np.mean(mpr_small)
print "avg recall:", np.mean(mrec_small)

avg f1: 0.733712879635
avg roc_auc: 0.822723640974
avg accuracy: 0.735695604763
avg precision: 0.738551109879
avg recall: 0.729625225378


In [222]:
ans_scores2 = []
for i in np.arange(0, 1.005, 0.01):
    y_pr = [a[:,1] * i  + (1-i) * b for a, b in zip(f_rf_prb, log_y_tst_prb)]
    ans_scores2.append([i, 1-i, [roc_auc_score(b, a) for a, b in zip(y_pr, y_tests)]])

In [229]:
b = sorted(ans_scores2, key = lambda x: -np.array(x[2]).mean())[0]
print "auc-mix:", np.mean(b[2])

auc-mix: 0.83804054977


In [246]:
def get_and(a, b):
    y = []
    for i, j in zip(a, b):
        if (i == 1) and (j == 1):
            y.append(1)
        else:
            y.append(0)
    return y

def get_or(a, b):
    y = []
    for i, j in zip(a, b):
        if (i == 1) or (j == 1):
            y.append(1)
        else:
            y.append(0)
    return y

In [242]:
y_pr = [get_and(a, b) for a, b in zip(f_rf_prd, log_y_tst_prd)]
pr = [precision_score(b, a) for a, b in zip(y_pr, y_tests)]
rec = [recall_score(b, a) for a, b in zip(y_pr, y_tests)]
f1 = [f1_score(b, a) for a, b in zip(y_pr, y_tests)]
acc = [accuracy_score(b, a) for a, b in zip(y_pr, y_tests)]

In [243]:
print "mix precision: ", np.mean(pr)
print "mix recall: ", np.mean(rec)
print "mix f1: ", np.mean(f1)
print "mix accuracy: ", np.mean(acc)

mix precision:  0.782735436921
mix recall:  0.634784098998
mix f1:  0.700494542161
mix accuracy:  0.729066473775


In [247]:
y_pr = [get_or(a, b) for a, b in zip(f_rf_prd, log_y_tst_prd)]
pr = [precision_score(b, a) for a, b in zip(y_pr, y_tests)]
rec = [recall_score(b, a) for a, b in zip(y_pr, y_tests)]
f1 = [f1_score(b, a) for a, b in zip(y_pr, y_tests)]
acc = [accuracy_score(b, a) for a, b in zip(y_pr, y_tests)]

In [248]:
print "mix precision: ", np.mean(pr)
print "mix recall: ", np.mean(rec)
print "mix f1: ", np.mean(f1)
print "mix accuracy: ", np.mean(acc)

mix precision:  0.669749612503
mix recall:  0.904914466224
mix f1:  0.769375592831
mix accuracy:  0.729208970332
